In [1]:
def readData(dataPath):
    dataFile = open(dataPath)
    dataStr = dataFile.read()
    dataList = dataStr.splitlines()
    return dataList

ordersPath = '/data/retail_db/orders/part-00000'
orders = readData(ordersPath)

orders[:10]



['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE',
 '6,2013-07-25 00:00:00.0,7130,COMPLETE',
 '7,2013-07-25 00:00:00.0,4530,COMPLETE',
 '8,2013-07-25 00:00:00.0,2911,PROCESSING',
 '9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT',
 '10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT']

In [2]:
def getOrderStatusCount(orders):
    StatusCount = {}
    for order in orders:
        orderAttribute = order.split(',')
        if StatusCount.get(orderAttribute[3]):
            StatusCount[orderAttribute[3]] += 1
        else:
            StatusCount[orderAttribute[3]] = 1
    return StatusCount

StatusCount = getOrderStatusCount(orders)

StatusCount
        

{'CLOSED': 7556,
 'PENDING_PAYMENT': 15030,
 'COMPLETE': 22899,
 'PROCESSING': 8275,
 'PAYMENT_REVIEW': 729,
 'PENDING': 7610,
 'ON_HOLD': 3798,
 'CANCELED': 1428,
 'SUSPECTED_FRAUD': 1558}

In [3]:
def getOrdersPerMonth(orders):
    MonthOrdersCount = {}
    for order in orders:
        orderAttribute = order.split(',')[1][:7]      
        if MonthOrdersCount.get(orderAttribute):
            MonthOrdersCount[orderAttribute] += 1
        else: 
            MonthOrdersCount[orderAttribute] = 1
    return MonthOrdersCount

MonthOrdersCount = getOrdersPerMonth(orders)
        
MonthOrdersCount

{'2013-07': 1533,
 '2013-08': 5680,
 '2013-09': 5841,
 '2013-10': 5335,
 '2013-11': 6381,
 '2013-12': 5892,
 '2014-01': 5908,
 '2014-02': 5635,
 '2014-03': 5778,
 '2014-04': 5657,
 '2014-05': 5467,
 '2014-06': 5308,
 '2014-07': 4468}

In [4]:
orderItemsPath = '/data/retail_db/order_items/part-00000'
orderItems = readData(orderItemsPath)

orderItems[:10]

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99',
 '6,4,365,5,299.95,59.99',
 '7,4,502,3,150.0,50.0',
 '8,4,1014,4,199.92,49.98',
 '9,5,957,1,299.98,299.98',
 '10,5,365,5,299.95,59.99']

In [5]:
def getSubtotalError(orderItems):
    ErrorOrders = []
    ErrorOrdersTuple = ()
    for orderItem in orderItems:
        orderItemAttribute = orderItem.split(',')
        SubTotal = round(float(orderItemAttribute[5]) * float(orderItemAttribute[3]),2)
        if SubTotal != float(orderItemAttribute[4]):
            ErrorOrders.append(orderItem)
            ErrorOrdersTuple += (SubTotal, orderItem)
    return (ErrorOrders, ErrorOrdersTuple)

ErrorOrders, ErrorOrdersTuple = getSubtotalError(orderItems)
ErrorOrders
#ErrorOrdersTuple
    
    

[]

In [6]:
def getOrderId(orderItems):
    orderIdDict = {}
    for orderItem in orderItems:
        orderItemAttribute = orderItem.split(',')
        orderIdDict[int(orderItemAttribute[1])] = None
    return orderIdDict
    
orderIdDict = getOrderId(orderItems)    
#orderIdDict
    


In [7]:
def getOrderNoItems(orders, orderIdDict):
    ordersNoItems = []
    for order in orders:
        orderAttribute = int(order.split(',')[0])
        if orderAttribute not in orderIdDict:
            ordersNoItems.append(order)
    return ordersNoItems


orderNoItems = getOrderNoItems(orders, orderIdDict)
orderNoItems[:10]

['3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '6,2013-07-25 00:00:00.0,7130,COMPLETE',
 '22,2013-07-25 00:00:00.0,333,COMPLETE',
 '26,2013-07-25 00:00:00.0,7562,COMPLETE',
 '32,2013-07-25 00:00:00.0,3960,COMPLETE',
 '40,2013-07-25 00:00:00.0,12092,PENDING_PAYMENT',
 '47,2013-07-25 00:00:00.0,8487,PENDING_PAYMENT',
 '53,2013-07-25 00:00:00.0,4701,PROCESSING',
 '54,2013-07-25 00:00:00.0,10628,PENDING_PAYMENT',
 '55,2013-07-25 00:00:00.0,2052,PENDING']

In [8]:
def getOrderIdAndDateDict(orders):
    orderIdAndDateDict = {}
    for order in orders:
        orderAttributes = order.split(',')
        orderIdAndDateDict[int(orderAttributes[0])] = orderAttributes[1]
    return orderIdAndDateDict

orderIdAndDateDict = getOrderIdAndDateDict(orders)

In [9]:
def getDailyProductRevenue(orderIdAndDateDict, orderItems):
    dailyProductRevenue = []
    for orderItem in orderItems:
        orderItemAttribute = orderItem.split(',')
        if orderIdAndDateDict.get(int(orderItemAttribute[1])):
            orderDate = orderIdAndDateDict[int(orderItemAttribute[1])]
            dailyProductRevenue += [[(orderDate,int(orderItemAttribute[2])),float(orderItemAttribute[4])]]

    return dailyProductRevenue

dailyProductRevenue = getDailyProductRevenue(orderIdAndDateDict, orderItems)

In [10]:
dailyProductRevenue[:10]

[[('2013-07-25 00:00:00.0', 957), 299.98],
 [('2013-07-25 00:00:00.0', 1073), 199.99],
 [('2013-07-25 00:00:00.0', 502), 250.0],
 [('2013-07-25 00:00:00.0', 403), 129.99],
 [('2013-07-25 00:00:00.0', 897), 49.98],
 [('2013-07-25 00:00:00.0', 365), 299.95],
 [('2013-07-25 00:00:00.0', 502), 150.0],
 [('2013-07-25 00:00:00.0', 1014), 199.92],
 [('2013-07-25 00:00:00.0', 957), 299.98],
 [('2013-07-25 00:00:00.0', 365), 299.95]]

In [11]:
def getDailyProductRevenue(orderIdAndDateDict, orderItems):
    dailyProductRevenue = {}
    for orderItem in orderItems:
        orderItemAttribute = orderItem.split(',')
        if orderIdAndDateDict.get(int(orderItemAttribute[1])):
            orderDate = orderIdAndDateDict[int(orderItemAttribute[1])]
            orderItemTuple = (orderDate + ' ' + orderItemAttribute[2], float(orderItemAttribute[4]))
            if dailyProductRevenue.get(orderItemTuple[0]):
                dailyProductRevenue[orderItemTuple[0]] += orderItemTuple[1]
            else:
                dailyProductRevenue[orderItemTuple[0]] = orderItemTuple[1]         
    return dailyProductRevenue

dailyProductRevenue = getDailyProductRevenue(orderIdAndDateDict, orderItems)

#dailyProductRevenue

In [20]:
dailyProductRevenue1000 = dailyProductRevenue.copy()
for item in dailyProductRevenue:
    if dailyProductRevenue.get(item) <= 1000:
        dailyProductRevenue1000.pop(item)
        
list(dailyProductRevenue1000.items())[:10]

[('2013-07-25 00:00:00.0 957', 9599.359999999993),
 ('2013-07-25 00:00:00.0 1073', 6999.649999999995),
 ('2013-07-25 00:00:00.0 502', 5100.0),
 ('2013-07-25 00:00:00.0 403', 5589.569999999994),
 ('2013-07-25 00:00:00.0 365', 7558.739999999994),
 ('2013-07-25 00:00:00.0 1014', 6397.439999999999),
 ('2013-07-25 00:00:00.0 191', 8499.149999999998),
 ('2013-07-25 00:00:00.0 627', 2879.2799999999997),
 ('2013-07-25 00:00:00.0 1004', 10799.459999999994),
 ('2013-07-26 00:00:00.0 403', 10529.189999999986)]